In [3]:
#웹 크롤링 라이브러리(beautifulsoup4) 설치 
!pip install beautifulsoup4
#selenium 설치
!pip install selenium
#webdriver-manager 설치
!pip install webdriver-manager

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ------------------------------------- -- 8.9/9.4 MB 46.3 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.8.30
    Uninstalling certifi-2024.8.30:
      Successfully uninstalled certifi-2024.8.30
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [4]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [5]:
# 데이터 저장 폴더 생성
import os
if not os.path.exists("crawled_data"):
    os.makedirs("crawled_data")

In [14]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Service 객체를 사용하여 드라이버 초기화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

In [15]:
import json

route = "./api_key.json"

# JSON 파일에서 API 키 읽기
with open(route, "r", encoding="utf-8") as file:
    api_keys = json.load(file)
try:
    client_id = api_keys["client_id"]
    client_secret = api_keys["client_secret"]
except KeyError:
    print("API 키가 JSON 파일에 없습니다. 확인 후 다시 시도하세요.")
    sys.exit(1)
else:
    print("API 키를 성공적으로 읽었습니다.")


API 키를 성공적으로 읽었습니다.


In [16]:
# selenium으로 검색 페이지 불러오기 #
naver_urls = []
postdate = []
titles = []

In [ ]:
keword = ['청주 성안길 식당','청주 율량동 식당','청주 복대동 식당', '청주 지웰시티 식당', '청주 오송 식당','청주 오창 식당','청주 가경동 식당','청주 사창동 식당', '청주 용암동 식당',
          '충주 식당', '제천 식당', '보은 식당', '옥천 식당', '영동 식당', '진천 식당', '괴산 식당', '음성 식당', '증평 식당', '단양 식당']

In [18]:
for word in keword:
    # selenium으로 검색 페이지 불러오기 #
    print(f"크롤링 시작: {word}")
    naver_urls = []
    postdate = []
    titles = []
    encText = urllib.parse.quote(word)
    end = 1
    if '청주' in word:
        display = 100
    else:        
        display = 90
    
    

    for start in range(end):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&start=" + str(start+1) + "&display=" + str(display) # JSON 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            
            data = json.loads(response_body.decode('utf-8'))['items']
            for row in data:
                if('blog.naver' in row['link']):
                    naver_urls.append(row['link'])
                    postdate.append(row['postdate'])
                    title = row['title']
                    # html태그제거
                    pattern1 = '<[^>]*>'
                    title = re.sub(pattern=pattern1, repl='', string=title)
                    titles.append(title)
            time.sleep(2)
        else:
            print("Error Code:" + rescode)
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from bs4 import BeautifulSoup
    import time

    contents = []  # 블로그 내용을 저장할 리스트

    try:
        for i, url in enumerate(naver_urls):
            print(f"블로그 URL ({i + 1}/{len(naver_urls)}): {url}")
            driver.get(url)
            time.sleep(3)  # 페이지 로딩 대기

            try:
                # iframe으로 전환
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "mainFrame"))
                )
                driver.switch_to.frame(iframe)

                # iframe 내부의 HTML 가져오기
                source = driver.page_source
                html = BeautifulSoup(source, "html.parser")

                # 블로그 내용 추출
                content = html.select_one("div.se-main-container")  # 스마트에디터 블로그
                if content:
                    content = content.get_text(strip=True)
                else:
                    # 일반 블로그 처리
                    content = html.select_one("div#postViewArea")
                    content = content.get_text(strip=True) if content else "내용을 가져올 수 없습니다."

                contents.append(content)

            except Exception as e:
                print(f"오류 발생: {e}")
                contents.append("error")  # 오류 발생 시 'error' 추가

            finally:
                # iframe에서 기본 컨텍스트로 전환
                driver.switch_to.default_content()

        # 리스트 길이 동기화 확인
        while len(contents) < len(titles):
            contents.append("error")  # 누락된 데이터를 'error'로 채움

        # 데이터프레임 생성
        news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
        news_df.to_csv(f'./crawled_data/{word}_blog.csv', index=False, encoding='utf-8-sig')

    except Exception as e:
        print(f"전체 오류 발생: {e}")

크롤링 시작: 청주 성안길 식당
블로그 URL (1/100): https://blog.naver.com/sb7056/223930476760
블로그 URL (2/100): https://blog.naver.com/who_u_who/223961590963
블로그 URL (3/100): https://blog.naver.com/cvcv5552/223398631493
블로그 URL (4/100): https://blog.naver.com/grb_080913/223891431394
블로그 URL (5/100): https://blog.naver.com/moon12_30/223862819753
블로그 URL (6/100): https://blog.naver.com/hakueiii/223838086075
블로그 URL (7/100): https://blog.naver.com/go_hy_go/223948818127
블로그 URL (8/100): https://blog.naver.com/2n_yang/223877114113
블로그 URL (9/100): https://blog.naver.com/satto0218/223850349702
블로그 URL (10/100): https://blog.naver.com/xksvptix1/223948932813
블로그 URL (11/100): https://blog.naver.com/dusnsl/223930716108
블로그 URL (12/100): https://blog.naver.com/gustjsdl8/223908879195
블로그 URL (13/100): https://blog.naver.com/ttpttl2002/223848063602
블로그 URL (14/100): https://blog.naver.com/sp2ces/223953770608
블로그 URL (15/100): https://blog.naver.com/hhhgee/223940675776
블로그 URL (16/100): https://blog.naver.com/rrsno

In [1]:
import pandas as pd
import glob
import os

folder = "./crawled_data"
files = glob.glob(os.path.join(folder, "*.csv"))

cheongju_files = [f for f in files if "청주" in os.path.basename(f)]
other_files = [f for f in files if "청주" not in os.path.basename(f)]

# 청주 파일 병합
df_cheongju = pd.concat([pd.read_csv(f, encoding="utf-8-sig") for f in cheongju_files], ignore_index=True)

# 그 외 파일 병합
df_other = pd.concat([pd.read_csv(f, encoding="utf-8-sig") for f in other_files], ignore_index=True)

# 결과 저장
df_cheongju.to_csv("./crawled_data/merged_cheongju.csv", index=False, encoding="utf-8-sig")
df_other.to_csv("./crawled_data/merged_other.csv", index=False, encoding="utf-8-sig")

print("청주 파일 병합 완료: merged_cheongju.csv")
print("그 외 파일 병합 완료: merged_other.csv")

청주 파일 병합 완료: merged_cheongju.csv
그 외 파일 병합 완료: merged_other.csv


In [ ]:
cheongju = pd.read_csv("./crawled_data/merged_cheongju.csv", encoding="utf-8-sig")
print(cheongju['date'].max())
print(cheongju['date'].min())

20250811
20131118


In [21]:
other = pd.read_csv("./crawled_data/merged_other.csv", encoding="utf-8-sig")
print(other['date'].max())
print(other['date'].min())

20250810
20060824


In [22]:
print(cheongju[cheongju['date'] == cheongju['date'].min()])
print(other[other['date'] == other['date'].min()])


                           title  \
280  폐업하고 업종변경함 한우데이 청주 사창동 정육식당   

                                               content      date  
280  폐업하고 다른음식점 생겼어요한우데이 청주 사창동 정육식당한우데이정육식당요즘 청주에는...  20131118  
                                               title  \
524  괴산/증평읍 한식집 &lt;우가촌&gt;우가한정식 &lt;음성식당&gt;뚝배기...    

                                               content      date  
524  우가촌증평읍과 괴산군내에서 규모가 가장 큰 한식 전문점으로 꼽는다. 한정식은 물론 ...  20060824  
